In [89]:
import pandas as pd

target=pd.read_excel('./2020.xlsx')


In [90]:
target.shape

(486, 37)

In [91]:
t_field=['科室','住院号','姓名','借支','还款1','还款2','还款3','入院时间','出院时间']
t_data=target[t_field]
t_data.fillna(0, inplace=True)

In [128]:
# t_data.isnull().sum()

dabing=pd.read_excel("./大病新表.xlsx",sheet_name="Sheet2")
dabing.fillna(0, inplace=True)
# s_field=['科室','住院号','姓名','借支金额','还款金额','还款金额.1','还款金额.2']
# dabing=fd[s_field]
# dabing.info()
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   科室      947 non-null    object 
 1   住院号     947 non-null    int64  
 2   姓名      947 non-null    object 
 3   借支日期    947 non-null    float64
 4   借支金额    947 non-null    float64
 5   还款日期    947 non-null    float64
 6   还款金额    947 non-null    float64
 7   还款日期.1  947 non-null    float64
 8   还款金额.1  947 non-null    float64
 9   还款日期.2  947 non-null    float64
 10  还款金额.2  947 non-null    float64
 11  批次      947 non-null    float64
dtypes: float64(9), int64(1), object(2)
memory usage: 88.9+ KB


In [93]:
# dabing['还款金额'] = pd.to_numeric(dabing['还款金额'],errors='coerce')
# dabing['还款金额'] = pd.to_numeric(dabing['还款金额'])

In [94]:
# dabing[dabing['还款金额'].isnull()]
# dabing['还款金额'].isnull().sum()
# dabing.info()

#### 查询住院号

In [95]:
value="822880"
dabing[dabing['住院号']==name]

,科室,住院号,姓名,借支金额,还款金额,还款金额.1,还款金额.2


#### 查询人名字

In [125]:
name="陈宾山"

In [127]:
# tmp=dabing[dabing['姓名']==name]
# tmp['批次']=5
# tmp

,科室,住院号,姓名,借支日期,借支金额,还款日期,还款金额,还款日期.1,还款金额.1,还款日期.2,还款金额.2,批次
78,创伤,820622,陈宾山,20200331.0,8187.01,20200331.0,1500.0,20200331.0,3811.35,0.0,0.0,5


In [122]:
t_data[t_data['姓名']==name]

,科室,住院号,姓名,借支,还款1,还款2,还款3,入院时间,出院时间
212,创伤,820622,陈宾山,8187.01,4111.35,0.0,0.0,2020.01.03,2020.01.22


In [142]:
import openpyxl
def add_sheet(data, excel_writer, sheet_name):
    """
    不改变原有Excel的数据，新增sheet。
    注：
        使用openpyxl操作Excel时Excel必需存在，因此要新建空sheet
        无论如何sheet页都会被新建，只是当sheet_name已经存在时会新建一个以1结尾的sheet，如：test已经存在时，新建sheet为test1，以此类推
    :param data: DataFrame数据
    :param excel_writer: 文件路径
    :param sheet_name: 新增的sheet名称
    :return:
    """
    book = openpyxl.load_workbook(excel_writer.path)
    excel_writer.book = book
    data.to_excel(excel_writer=excel_writer, sheet_name=sheet_name, index=None,header=True)
 
    excel_writer.close()

In [143]:
import process 
import time
result = pd.DataFrame(columns = t_data.columns) # 错误结果 
matched =  pd.DataFrame(columns = t_data.columns)  # 正确结果
drop_index=[] #记录需要删除的行号

max_steps=t_data.shape[0]
process_bar = process.ShowProcess(max_steps, 'OK')
for idx,row in t_data.iterrows():
    # print(type(row))
    # print(row["住院号"])
    fil=(dabing["住院号"]==row["住院号"]) \
        &(dabing["姓名"]==row["姓名"]) \
        &(dabing["借支金额"]==row["借支"]) \
        &(dabing["还款金额"]==row["还款1"]) \
        &(dabing["还款金额.1"]==row["还款2"]) \
        &(dabing["还款金额.2"]==row["还款3"]) 
        # &(fd["科室"]==row["科室"]) \

    _temp= dabing[fil]  
    if len(_temp)==0: # can not match
        # print("{} is false".format(idx))
        result.loc[len(result)] = row
    elif len(_temp)==1:
        # print("{} is true,fd index is {}".format(idx,_temp.index.tolist()))
        matched.loc[len(matched)] = row
        drop_index.extend(_temp.index.tolist())
        dabing.loc[_temp.index,'批次']=5
        # print(dabing.loc[_temp.index,'批次'])
    else:
        print("有两条记录满足{}".format(_temp.index))
    process_bar.show_process()
    time.sleep(0.01)
# print(result)
# write to error
result.to_excel("./result.xlsx",sheet_name="Sheet1",index=False)
matched.to_excel("./matched.xlsx",sheet_name="Sheet1",index=False)
# print(drop_index)
left=fd.drop(drop_index) # 剩余对比结果
left.to_excel("./left.xlsx",sheet_name="Sheet1",index=False)
excel_writer = pd.ExcelWriter("./大病新表.xlsx", engine='openpyxl')
add_sheet(dabing,excel_writer,"比对")
print("对比结果如下：")
print("-----------------------------------")
print("一共对比记录："+str(target.shape[0]))
print("其中，正确比对合计："+str(matched.shape[0]))
print("错误对比结果合计："+str(result.shape[0]))
print("原始结果合计："+str(fd.shape[0]))
print("剩余对比结果合计："+str(left.shape[0]))
print("-----------------------------------")
print("最后产生三个文件：")
print("错误结果：result.xlsx")
print("正确结果：matched.xlsx")
print("剩余结果：left.xlsx")
print("-----------------------------------")


OK
对比结果如下：
-----------------------------------
一共对比记录：486
其中，正确比对合计：354
错误对比结果合计：132
原始结果合计：947
剩余对比结果合计：593
-----------------------------------
最后产生三个文件：
错误结果：result.xlsx
正确结果：matched.xlsx
剩余结果：left.xlsx
-----------------------------------
